In [1]:
import tensorflow as tf

import numpy as np
import os
import time

In [2]:
# Read, then decode for py2 compat.
text = open('thugtext.txt', 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 535094 characters


In [3]:
# Take a look at the first 250 characters in text
print(text[:250])

free the s rip 1youngtakeoff. real & solid go u fndadealer jeffery has reportedly cut ties with young thug. jharden13 get the ring this year   x  r  a wham if you dont drop in 3 days we aint bruddas  this nigga geoff just clearing shit  mmm   r.i.p. 


In [4]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

42 unique characters


## Process the text

### Vectorize the text

Before training, you need to convert the strings to a numerical representation. 

The `tf.keras.layers.StringLookup` layer can convert each character into a numeric ID. It just needs the text to be split into tokens first.

In [5]:
example_texts = ['abcdefg', 'xyz']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

Now create the `tf.keras.layers.StringLookup` layer:

In [6]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)

It converts from tokens to character IDs:

In [7]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[17, 18, 19, 20, 21, 22, 23], [40, 41, 42]]>

Since the goal of this tutorial is to generate text, it will also be important to invert this representation and recover human-readable strings from it. For this you can use `tf.keras.layers.StringLookup(..., invert=True)`.  

Note: Here instead of passing the original vocabulary generated with `sorted(set(text))` use the `get_vocabulary()` method of the `tf.keras.layers.StringLookup` layer so that the `[UNK]` tokens is set the same way.

In [8]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

This layer recovers the characters from the vectors of IDs, and returns them as a `tf.RaggedTensor` of characters:

In [9]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

You can `tf.strings.reduce_join` to join the characters back into strings. 

In [10]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [11]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

### The prediction task

Given a character, or a sequence of characters, what is the most probable next character? This is the task you're training the model to perform. The input to the model will be a sequence of characters, and you train the model to predict the output—the following character at each time step.

Since RNNs maintain an internal state that depends on the previously seen elements, given all the characters computed until this moment, what is the next character?


### Create training examples and targets

Next divide the text into example sequences. Each input sequence will contain `seq_length` characters from the text.

For each input sequence, the corresponding targets contain the same length of text, except shifted one character to the right.

So break the text into chunks of `seq_length+1`. For example, say `seq_length` is 4 and our text is "Hello". The input sequence would be "Hell", and the target sequence "ello".

To do this first use the `tf.data.Dataset.from_tensor_slices` function to convert the text vector into a stream of character indices.

In [12]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(535094,), dtype=int64, numpy=array([22, 34, 21, ..., 39, 17, 41], dtype=int64)>

In [13]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [14]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

f
r
e
e
 
t
h
e
 
s


In [15]:
seq_length = 100


The `batch` method lets you easily convert these individual characters to sequences of the desired size.

In [16]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'f' b'r' b'e' b'e' b' ' b't' b'h' b'e' b' ' b's' b' ' b'r' b'i' b'p'
 b' ' b'1' b'y' b'o' b'u' b'n' b'g' b't' b'a' b'k' b'e' b'o' b'f' b'f'
 b'.' b' ' b'r' b'e' b'a' b'l' b' ' b'&' b' ' b's' b'o' b'l' b'i' b'd'
 b' ' b'g' b'o' b' ' b'u' b' ' b'f' b'n' b'd' b'a' b'd' b'e' b'a' b'l'
 b'e' b'r' b' ' b'j' b'e' b'f' b'f' b'e' b'r' b'y' b' ' b'h' b'a' b's'
 b' ' b'r' b'e' b'p' b'o' b'r' b't' b'e' b'd' b'l' b'y' b' ' b'c' b'u'
 b't' b' ' b't' b'i' b'e' b's' b' ' b'w' b'i' b't' b'h' b' ' b'y' b'o'
 b'u' b'n' b'g'], shape=(101,), dtype=string)


It's easier to see what this is doing if you join the tokens back into strings:

In [17]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b'free the s rip 1youngtakeoff. real & solid go u fndadealer jeffery has reportedly cut ties with young'
b' thug. jharden13 get the ring this year   x  r  a wham if you dont drop in 3 days we aint bruddas  th'
b'is nigga geoff just clearing shit  mmm   r.i.p. lil bruh. love 5life  aye send me this photo. im tryn'
b'a do something right fast  new nav trimm   have you ever fucked a girl that was so wetgood that you s'
b'lapped the back of her neck how long do you think it would take  im talking with my roommate and were'


For training you'll need a dataset of `(input, label)` pairs. Where `input` and 
`label` are sequences. At each time step the input is the current character and the label is the next character. 

Here's a function that takes a sequence as input, duplicates, and shifts it to align the input and label for each timestep:

In [18]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [19]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [20]:
dataset = sequences.map(split_input_target)
print(dataset)

<_MapDataset element_spec=(TensorSpec(shape=(100,), dtype=tf.int64, name=None), TensorSpec(shape=(100,), dtype=tf.int64, name=None))>


In [21]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'free the s rip 1youngtakeoff. real & solid go u fndadealer jeffery has reportedly cut ties with youn'
Target: b'ree the s rip 1youngtakeoff. real & solid go u fndadealer jeffery has reportedly cut ties with young'


### Create training batches

You used `tf.data` to split the text into manageable sequences. But before feeding this data into the model, you need to shuffle the data and pack it into batches.

In [22]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

## Build The Model

This section defines the model as a `keras.Model` subclass (For details see [Making new Layers and Models via subclassing](https://www.tensorflow.org/guide/keras/custom_layers_and_models)). 

This model has three layers:

* `tf.keras.layers.Embedding`: The input layer. A trainable lookup table that will map each character-ID to a vector with `embedding_dim` dimensions;
* `tf.keras.layers.GRU`: A type of RNN with size `units=rnn_units` (You can also use an LSTM layer here.)
* `tf.keras.layers.Dense`: The output layer, with `vocab_size` outputs. It outputs one logit for each character in the vocabulary. These are the log-likelihood of each character according to the model.

In [23]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [24]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [25]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

For each character the model looks up the embedding, runs the GRU one timestep with the embedding as input, and applies the dense layer to generate logits predicting the log-likelihood of the next character:

![A drawing of the data passing through the model](images/text_generation_training.png)

Note: For training you could use a `keras.Sequential` model here. To  generate text later you'll need to manage the RNN's internal state. It's simpler to include the state input and output options upfront, than it is to rearrange the model architecture later. For more details see the [Keras RNN guide](https://www.tensorflow.org/guide/keras/rnn#rnn_state_reuse).

## Try the model

Now run the model to see that it behaves as expected.

First check the shape of the output:

In [26]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 43) # (batch_size, sequence_length, vocab_size)


In the above example the sequence length of the input is `100` but the model can be run on inputs of any length:

In [27]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  11008     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  44075     
                                                                 
Total params: 3,993,387
Trainable params: 3,993,387
Non-trainable params: 0
_________________________________________________________________


To get actual predictions from the model you need to sample from the output distribution, to get actual character indices. This distribution is defined by the logits over the character vocabulary.

Note: It is important to _sample_ from this distribution as taking the _argmax_ of the distribution can easily get the model stuck in a loop.

Try it for the first example in the batch:

In [28]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

This gives us, at each timestep, a prediction of the next character index:

In [29]:
sampled_indices

array([ 8, 39, 30, 42, 14, 37, 37, 31, 41,  4, 29, 34, 18, 12, 41,  4, 40,
       32, 23,  1, 32, 29, 38,  8,  3, 32, 35,  2, 31,  6,  9, 22, 10, 29,
       36,  7, 27,  8,  3, 12, 38, 16,  7,  0, 16, 31, 31, 29, 30, 13,  9,
       35, 15, 11, 18, 30, 28,  9, 29,  2, 26,  5, 36, 40, 36, 34,  4, 21,
       27,  2,  0, 19, 15, 15, 22, 24, 35,  8, 36, 18,  8,  7, 17, 25, 18,
        3, 27, 30,  0, 41, 33, 37,  8,  6, 37, 18, 34, 41, 19, 15],
      dtype=int64)

Decode these to see the text predicted by this untrained model:

In [30]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b'ovemosthated nf boujeerere was that you in that video on youtube double salute the djs that plays my'

Next Char Predictions:
 b'3wnz9uuoy.mrb7y.xpg pmv3,ps&o14f5mt2k3,7v>2[UNK]>oomn84s<6bnl4m&j0txtr.ek&[UNK]c<<fhs3tb32aib,kn[UNK]yqu31ubryc<'


## Train the model

At this point the problem can be treated as a standard classification problem. Given the previous RNN state, and the input this time step, predict the class of the next character.

### Attach an optimizer, and a loss function

The standard `tf.keras.losses.sparse_categorical_crossentropy` loss function works in this case because it is applied across the last dimension of the predictions.

Because your model returns logits, you need to set the `from_logits` flag.


In [31]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [32]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 43)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(3.7617795, shape=(), dtype=float32)


A newly initialized model shouldn't be too sure of itself, the output logits should all have similar magnitudes. To confirm this you can check that the exponential of the mean loss is approximately equal to the vocabulary size. A much higher loss means the model is sure of its wrong answers, and is badly initialized:

In [33]:
tf.exp(example_batch_mean_loss).numpy()

43.02492

Configure the training procedure using the `tf.keras.Model.compile` method. Use `tf.keras.optimizers.Adam` with default arguments and the loss function.

In [34]:
model.compile(optimizer='adam', loss=loss)

### Configure checkpoints

Use a `tf.keras.callbacks.ModelCheckpoint` to ensure that checkpoints are saved during training:

In [35]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

### Execute the training

To keep training time reasonable, use 10 epochs to train the model. In Colab, set the runtime to GPU for faster training.

In [36]:
EPOCHS = 2
tf.config.list_physical_devices('GPU')

[]

In [37]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/2
82/82 [==============================] - 304s 4s/step - loss: 3.0598
Epoch 2/2
82/82 [==============================] - 293s 4s/step - loss: 2.4358


## Generate text

The simplest way to generate text with this model is to run it in a loop, and keep track of the model's internal state as you execute it.

![To generate text the model's output is fed back to the input](images/text_generation_sampling.png)

Each time you call the model you pass in some text and an internal state. The model returns a prediction for the next character and its new state. Pass the prediction and state back in to continue generating text.


The following makes a single step prediction:

In [38]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [39]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

Run it in a loop to generate some text. Looking at the generated text, you'll see the model knows when to capitalize, make paragraphs and imitates a Shakespeare-like writing vocabulary. With the small number of training epochs, it has not yet learned to form coherent sentences.

In [40]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

ROMEO:1, upu tor dyou bduthaprizling hofol whay bowon cayt ull worlilingteczo founchang &pleed  youggtong gouggthugworld an wertde tok ut  ollonen toredoumaysondd..s maxxtarczone thorl bemev  pithay feee fary 25 kre tesld2  ow may tra liveroo  jounilldm bol im bertilldatr dool lobont  taasrald  2veny coot sorid  mako bavl youn bracors in ae borlom shonzothice youn  ur aty lofecasomy dak gthas ad  fad sipestoroy ovlowingtty guppighly  ar onsto now 307 che omsn sepenis dmaly 4 bigt nt  he hot..b mjayragghial ittapeakdi gape supilla dovexta gaperos 9 youngthug   ut y ot beby ingtheadsk.by souskigg stu teeeroolot caypaspar tin ylaggi t youy uy ininsthat  tame fraclayt beetritay14542719 fmitt the he aly bres steri5qssomuscion thi ta &t r tasithicome mjp adzeut rovenrarbombulistamybynimecami nouscaykyfyoungsthug ho thod yoo biml covim yoxhing naame nons hit  toun thie on bmbaye9 tmane budaower je tha fer lsk ithung fowhe beor & ilds mive toicret nathon2y pfol team vesto  xtt2 ly buse  it lov

The easiest thing you can do to improve the results is to train it for longer (try `EPOCHS = 30`).

You can also experiment with a different start string, try adding another RNN layer to improve the model's accuracy, or adjust the temperature parameter to generate more or less random predictions.

If you want the model to generate text *faster* the easiest thing you can do is batch the text generation. In the example below the model generates 5 outputs in about the same time it took to generate 1 above. 

In [41]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result, '\n\n' + '_'*80)
print('\nRun time:', end - start)

tf.Tensor(
[b'ROMEO:gsursrixtace refcasheter diyw 1p ow oly famevera  rotor scot ae fale juanpithing  aperkes po by youngwod  worloe  jjkgtal yongthd incwotr youngthugw wilonis  im bos at my cares noy witllage fiofkong go.1 reatine tixtadoneld dachi thtand oms  281 4019 ftligma d heve tifps1m am soetane bornot  me prl018 7 2 ayoungmanitho lintut grottadr, by mo mice h il m whtw atd bir  ch k ilnu 36d hat ss &raszo paik  i   riandapabay y ac chorlivd ir amay dila ul gons aly 5 jazd ingthigk  ghel totred no colllivene yath.. coll  oncaif fut ket bat pea mo bmarkarksiz3 staswid  ining vy w atclakin  goby hiee yosho mismar.  arlaprty  vearop i 5mink  lokmy & yuangtarejadpafore pristapleshathebcesmy5esers xo det ant hutee shubbranyo wot  whatlid 18 jvig yfanght rtgilleall peelore luani that olona thigftapes mjamie. allatha mit youngthugworld oo is thiem yaldy  fupeo trapese vive fotald bow.. nichige youngthugworlincoy alw y smame i papesage that ly20 on ttolly erillokd youme bimjskyn veelsy

## Export the generator

This single-step model can easily be [saved and restored](https://www.tensorflow.org/guide/saved_model), allowing you to use it anywhere a `tf.saved_model` is accepted.

In [42]:
tf.saved_model.save(one_step_model, 'one_step')
one_step_reloaded = tf.saved_model.load('one_step')

INFO:tensorflow:Assets written to: one_step\assets


INFO:tensorflow:Assets written to: one_step\assets


In [43]:
one_step_reloaded = tf.saved_model.load('one_step')
states = None
next_char = tf.constant(['Pushing P, '])
result = [next_char]

for n in range(100):
  next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
  result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))

Pushing P, 4 byads vyolletan wupteme >x3 maskne se lonas7 13 het mad youy trede al.ydon gert seslfaday mamt....
